### Data extraction

In [26]:
from utils import fetch_log_returns
training_dataset = fetch_log_returns(start = '1995-01-01', end ='1995-12-31')

[*********************100%***********************]  5 of 5 completed


In [77]:
training_dataset.values

array([[-8.03420318e-04, -1.03968302e-02, -6.34465166e-03,
        -3.45697194e-03, -1.95645036e-02],
       [ 7.38303962e-04,  3.28007496e-03,  1.07261458e-02,
        -4.93917085e-03,  0.00000000e+00],
       [ 3.25539389e-04, -2.33000766e-03, -3.00612927e-03,
        -3.82611528e-03, -1.43458320e-02],
       ...,
       [ 3.74407369e-04, -1.62287613e-04,  4.93541396e-03,
         5.36316418e-03, -2.68870072e-02],
       [-6.67454827e-04, -2.87200507e-03,  8.16115201e-05,
        -6.95147773e-03,  1.73196363e-02],
       [ 2.94296798e-03, -5.82632554e-03,  3.42115394e-03,
        -2.50644600e-04,  8.74144700e-03]])

### Define the problem instance parameters

In [142]:
K = 1 # number of qubits assigned per asset 

NUM_ASSETS = len(training_dataset.columns) # number of assets
N = NUM_ASSETS*K # number of total qubits
NLAYERS = 2 # number of layers of the ansatz

### Create the Ansatz

In [146]:
from qibo import models, gates

def build_ansatz_circuit(parameters):
    c = models.Circuit(N)
    for _ in range(NLAYERS):
        c.add((gates.RY(q, theta=0) for q in range(N)))
        c.add((gates.CZ(q, q+1) for q in range(0, N-1, 2)))
        c.add((gates.RY(q, theta=0) for q in range(N)))
        c.add((gates.CZ(q, q+1) for q in range(1, N-2, 2)))
        c.add(gates.CZ(0, N-1))
    c.add((gates.RY(q, theta=0) for q in range(N)))
    c.set_parameters(parameters)
    c.add(gates.M(0,1,2,3,4))
    print(c.draw())
    return c

### Building the Portfolio Optimization Hamiltonian

In [147]:

# Define the vector that contains the pauli opeations
def A(i: int, bit_string: list):
    return sum(2 ** (k - 2) * ((1 - bit_string[k + i*K]) / 2) for k in range(K))

# Return term
lambda_1 = 1.0
bit_string = [1,1,0,0,1]

def return_term(dataset,bit_string):
    h1=0
    for i, asset in enumerate(dataset.columns):
        h1 += A(i,bit_string) * dataset[asset].values
    return (-1)*sum(h1)
print('Return term:', return_term(training_dataset,bit_string))


# Volatility term
lambda_2 = 1.0 
sigma_target = 0.2

def tilde_sigma(i,j):
    if i==j: 
        return training_dataset.cov().values[i][j]
    elif i<j:
        return 2 * training_dataset.cov().values[i][j]
    else: 
        return 0
        
        

def risk_term(dataset):
    h2 = 0
    for i in range(NUM_ASSETS):
        for j in range(NUM_ASSETS):  
            h2 += tilde_sigma(i,j) * A(i) * A(i) - sigma_target ** 2

    return h2 ** 2

# # Third term: lambda_3 * (sum(A_i) - 1)^2
# term3 = lambda_3 * (sum(A[i] for i in range(N)) - 1) ** 2



Return term: -0.024885066923000843


In [141]:
import numpy as np
from qibo.optimizers import optimize
    
def energy(parameters, dataset):
    
    circuit = build_ansatz_circuit(parameters)
    
    # Measure the qubits
    result = circuit(nshots=1)
    
    return return_term(dataset, result.samples(binary=True)[0])




# Optimize starting from a random guess for the variational parameters
x0 = np.random.uniform(0, 2*np.pi, 2*M*NLAYERS + M)
# data = np.random.normal(0, 1, size=2**nqubits)

# perform optimization
best, params, extra = optimize(energy, x0, args=(training_dataset))

# set final solution to circuit instance
print('Ground state:', best)

print('Optimal parameters', params)

print('Optimization process info', extra)

q0: ─RY─o───RY───o─RY─o───RY───o─RY─M─
q1: ─RY─Z───RY─o─|─RY─Z───RY─o─|─RY─M─
q2: ─RY───o─RY─Z─|─RY───o─RY─Z─|─RY─M─
q3: ─RY───Z─RY───|─RY───Z─RY───|─RY─M─
q4: ─RY─────RY───Z─RY─────RY───Z─RY─M─
q0: ─RY─o───RY───o─RY─o───RY───o─RY─M─
q1: ─RY─Z───RY─o─|─RY─Z───RY─o─|─RY─M─
q2: ─RY───o─RY─Z─|─RY───o─RY─Z─|─RY─M─
q3: ─RY───Z─RY───|─RY───Z─RY───|─RY─M─
q4: ─RY─────RY───Z─RY─────RY───Z─RY─M─
q0: ─RY─o───RY───o─RY─o───RY───o─RY─M─
q1: ─RY─Z───RY─o─|─RY─Z───RY─o─|─RY─M─
q2: ─RY───o─RY─Z─|─RY───o─RY─Z─|─RY─M─
q3: ─RY───Z─RY───|─RY───Z─RY───|─RY─M─
q4: ─RY─────RY───Z─RY─────RY───Z─RY─M─
q0: ─RY─o───RY───o─RY─o───RY───o─RY─M─
q1: ─RY─Z───RY─o─|─RY─Z───RY─o─|─RY─M─
q2: ─RY───o─RY─Z─|─RY───o─RY─Z─|─RY─M─
q3: ─RY───Z─RY───|─RY───Z─RY───|─RY─M─
q4: ─RY─────RY───Z─RY─────RY───Z─RY─M─
q0: ─RY─o───RY───o─RY─o───RY───o─RY─M─
q1: ─RY─Z───RY─o─|─RY─Z───RY─o─|─RY─M─
q2: ─RY───o─RY─Z─|─RY───o─RY─Z─|─RY─M─
q3: ─RY───Z─RY───|─RY───Z─RY───|─RY─M─
q4: ─RY─────RY───Z─RY─────RY───Z─RY─M─
q0: ─RY─o───RY───o─RY─o──